# Emotion Recognition with BERT
Setting Environment

In [ ]:
# install ktrain on Google Colab
!pip3 install ktrain

In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

## 1. Import Data

In [ ]:
data_train = pd.read_csv('https://raw.githubusercontent.com/lukasgarbas/nlp-text-emotion/master/data/data_train.csv', encoding='utf-8')
data_test = pd.read_csv('https://raw.githubusercontent.com/lukasgarbas/nlp-text-emotion/master/data/data_test.csv', encoding='utf-8')

X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

print(data.head(10))

In [ ]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

## 2. Data preprocessing

* BERT can handle a maximum length of 512

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

## 2. Training and validation


Loading the pretrained BERT for text classification 

In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Wrap it in a Learner object

In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

Fine Tuning for 5 epochs

In [ ]:
learner.fit_onecycle(2e-5, 5)

Validation

In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

#### Testing with other inputs

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

In [ ]:
message = 'I just lost my job.'
prediction = predictor.predict(message)
print('predicted:'prediction)

message = 'What an amazing weather it is today.'
prediction = predictor.predict(message)
print('predicted:'prediction)

message = 'I was kicked out of my home at night'
prediction = predictor.predict(message)
print('predicted:'prediction)

message = 'I went for a 5Km walk.'
prediction = predictor.predict(message)
print('predicted:'prediction)

## 4. Saving Bert model


In [ ]:
# let's save the predictor for later use
predictor.save("bert_model")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!ls /content/gdrive/MyDrive

In [ ]:
path = F"/content/gdrive/MyDrive/bert_model"
predictor.save(path)

Done! For inference, use: ktrain.load_predictor